---
# ライブラリのインポート
---
~~~python
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import from_numpy, manual_seed
from torch.utils.data import DataLoader, TensorDataset
~~~

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import from_numpy, manual_seed
from torch.utils.data import DataLoader, TensorDataset

---
# 行列の形状を確認し、変形
---
~~~python
X_train, X_test, y_train, y_test = np.load("./animal.npy" , allow_pickle=True)

print("変形前:", X_train.shape)

# 形状を（件数、縦、横、RGB）から（件数、RGB、縦、横）に変換する
X_train = np.transpose(X_train, (0, 3, 1, 2))
X_test = np.transpose(X_test, (0, 3, 1, 2))

print("変更後:", X_train.shape)
~~~

In [2]:
X_train, X_test, y_train, y_test = np.load("./animal.npy" , allow_pickle=True)

print("変形前:", X_train.shape)

# 形状を（件数、縦、横、RGB）から（件数、RGB、縦、横）に変換する
X_train = np.transpose(X_train, (0, 3, 1, 2))
X_test = np.transpose(X_test, (0, 3, 1, 2))

print("変更後:", X_train.shape)

変形前: (420, 50, 50, 3)
変更後: (420, 3, 50, 50)


---
# テンソル型に変換し、データローダにセット
---
~~~python
#numpyのデータをテンソル型に変換
X_train = from_numpy(X_train.astype(np.float32))
y_train = from_numpy(y_train.astype(np.int64))
X_test = from_numpy(X_test.astype(np.float32))
y_test = from_numpy(y_test.astype(np.int64))

#テンソル型のXとYをデータローダーにセット
train = TensorDataset(X_train, y_train)

# PyTorch のRNGシードを初期化（再現性の為）
manual_seed(0)

#15件づつに分割してシャッフル
train_loader = DataLoader(train, batch_size=15, shuffle=True)
~~~

In [3]:
#numpyのデータをテンソル型に変換
X_train = from_numpy(X_train.astype(np.float32))
y_train = from_numpy(y_train.astype(np.int64))
X_test = from_numpy(X_test.astype(np.float32))
y_test = from_numpy(y_test.astype(np.int64))

#テンソル型のXとYをデータローダーにセット
train = TensorDataset(X_train, y_train)

# PyTorch のRNGシードを初期化（再現性の為）
manual_seed(0)

#15件づつに分割してシャッフル
train_loader = DataLoader(train, batch_size=15, shuffle=True)

---
# 学習モデルを定義
---
~~~python
# CNNを実装する
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
~~~

In [4]:
# CNNを実装する
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

---
# 機械学習実行
---
~~~python
model = Net()
# 交差エントロピー
criterion = nn.CrossEntropyLoss()
# 確率的勾配降下法
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(500): #学習回数500回
    total_loss = 0
     
    for X_train, y_train in train_loader:
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    #途中の損失関数を確認
    if (epoch + 1) % 50 == 0:
        print(epoch + 1, total_loss)
~~~

In [5]:
model = Net()
# 交差エントロピー
criterion = nn.CrossEntropyLoss()
# 確率的勾配降下法
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(500): #学習回数500回
    total_loss = 0

    for X_train, y_train in train_loader:
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    #途中の損失関数を確認
    if (epoch + 1) % 50 == 0:
        print(epoch + 1, total_loss)

50 0.1258764510130277
100 0.0016908556669932295
150 0.0007928972752324626
200 0.0005082241806917409
250 0.00036980733335667537
300 0.0002892360066155675
350 0.00023633024159153138
400 0.0001990537340184062
450 0.00017158258067695442
500 0.00015034916627598705


---
# 学習済みモデルの保存
---
~~~python
# モデルを保存する
PATH = './animal_cnn.pth'
torch.save(model.state_dict(), PATH)
~~~

In [6]:
# モデルを保存する
PATH = './animal_cnn.pth'
torch.save(model.state_dict(), PATH)